In [1]:
import pandas as pd
from xgboost import XGBClassifier

# Load the model with the same parameters as in your training code
best_params = {'learning_rate': 0.2242647022315775, 'n_estimators': 79, 'max_depth': 7, 'min_child_weight': 1, 'subsample': 0.8927217522311153, 'colsample_bytree': 0.7750851408245816, 'gamma': 0.4593480844032318, 'reg_alpha': 0.4090488353340545, 'reg_lambda': 0.08331747739058343, 'scale_pos_weight': 8.196181010536847}#{'learning_rate': 0.12107949809157363, 'n_estimators': 139, 'max_depth': 4, 'min_child_weight': 1, 'subsample': 0.9209752945123717, 'colsample_bytree': 0.8547946386134507, 'gamma': 0.3034587970270446, 'reg_alpha': 0.594892621788095, 'reg_lambda': 0.6783176414022941, 'scale_pos_weight': 3.137907077722744}
model = XGBClassifier(**best_params, random_state=42, eval_metric='logloss')

# Define the features in the same order as used in training
features = [
    'CumulativePlayoffProgScore', '3P%', 'FT%',
    'OFFRTG', 'DEFRTG', 'AST RATIO',
    'REB%', 'TOV%', 'PACE', 'AvgPIE_NextYearPlayers', 'Performance_NextYearCoach'
]

# Load and prepare the training data
dataset = pd.read_csv('./cleaned_data/dataset.csv')
X_train = dataset[features]
y_train = dataset['PlayoffNextSeason']

# Train the model on all training data
model.fit(X_train, y_train)

# Load year 10 data
year_10_data = pd.read_csv('./cleaned_data/year_10_stats.csv')

# Prepare the prediction data
X_pred = year_10_data[features]

# Get predictions and probabilities
predictions = model.predict(X_pred)
probabilities = model.predict_proba(X_pred)

# Create results DataFrame
results = pd.DataFrame({
    'Team': year_10_data['Team'],
    'Playoff_Prediction': predictions,
    'Playoff_Probability': probabilities[:, 1]  # Probability of positive class (making playoffs)
})

# Sort by probability in descending order
results = results.sort_values('Playoff_Probability', ascending=False)

# Display results
print("\nPlayoff Predictions for Next Season:")
print("=====================================")
for _, row in results.iterrows():
    print(f"{row['Team']}: {row['Playoff_Probability']*100:.2f}% chance of making playoffs")

# Save to CSV
results.to_csv('playoff_predictions_year_10.csv', index=False)
print("\nResults have been saved to 'playoff_predictions_year_10.csv'")


Playoff Predictions for Next Season:
LAS: 99.70% chance of making playoffs
SAS: 99.43% chance of making playoffs
IND: 99.37% chance of making playoffs
SEA: 98.15% chance of making playoffs
PHO: 89.42% chance of making playoffs
NYL: 89.42% chance of making playoffs
MIN: 84.52% chance of making playoffs
CON: 73.37% chance of making playoffs
WAS: 61.21% chance of making playoffs
ATL: 25.11% chance of making playoffs
CHI: 5.92% chance of making playoffs
TUL: 0.33% chance of making playoffs

Results have been saved to 'playoff_predictions_year_10.csv'
